Le logiciel Copilote, développé par la société Infologic, est un progiciel de gestion intégré (ERP) destiné aux entreprises du secteur agroalimentaire. Il permet de centraliser et de piloter l’ensemble des activités d’une entreprise comme la production, la logistique ou encore la qualité, la finance et la gestion commerciale. L’entreprise Infologic, experte en solutions numériques pour l’agroalimentaire depuis plus de trente ans, accompagne ses clients dans leur digitalisation et leur optimisation de la chaîne de valeur. Chaque jour, des centaines d’utilisateurs interagissent avec Copilote, générant ainsi un grand volume de traces d’utilisation reflétant leurs comportements et leurs habitudes de travail. L’objectif de ce projet est d’exploiter ces traces pour identifier automatiquement les utilisateurs à partir de leurs actions. Cette problématique mélange l’analyse de données, la modélisation comportementale et la classification automatique, la data science semble donc etre le domaine le plus adequat pour la résolution de ce problème. En mobilisant des méthodes de traitement et d’apprentissage supervisé, nous allons chercher à extraire des informations pertinentes à partir des logs et à construire un modèle capable de reconnaître un utilisateur en fonction de ses interactions avec le logiciel.

## Cell 1: Setup & Imports

Load the raw CSV into a DataFrame.

In [2]:
from construct.create_df import read_ds

features_train = read_ds("train")
features_test = read_ds("test")
features_train.shape, features_test.shape

((3279, 14470), (324, 7726))

In [3]:
features_train.head()

,util,navigateur,col3,col4,col5,col6,col7,col8,col9,col10,...,col14461,col14462,col14463,col14464,col14465,col14466,col14467,col14468,col14469,col14470
0,nuh,Firefox,Création d'un écran(infologic.core.accueil.Acc...,Affichage d'une dialogue,Exécution d'un bouton,Fermeture d'une dialogue,Affichage d'une dialogue,Exécution d'un bouton,Fermeture d'une dialogue,Création d'un écran(infologic.core.gui.control...,...,None,None,None,None,None,None,None,None,None,None
1,muz,Google Chrome,Création d'un écran(infologic.core.gui.control...,Création d'un écran(infologic.core.gui.control...,t5,Sélection d’un onglet(infologic.orga.modules.O...,t10,Exécution d'un bouton,t15,Sélection d’un onglet,...,None,None,None,None,None,None,None,None,None,None
2,zrx,Microsoft Edge,Affichage d'une dialogue(infologic.core.gui.co...,Exécution d'un bouton,Chainage,Fermeture d'une dialogue,Affichage d'une dialogue(infologic.acti.module...,Clic sur une grille d'historique de recherche,Raccourci,Fermeture d'une dialogue,...,None,None,None,None,None,None,None,None,None,None
3,pou,Firefox,Création d'un écran(infologic.core.gui.control...,t5,Exécution d'un bouton(MAINT),Affichage d'une dialogue,Fermeture d'une dialogue,Double-clic,Exécution d'un bouton,Lancement d'une stat(infologic.core.gui.contro...,...,None,None,None,None,None,None,None,None,None,None
4,ald,Google Chrome,Affichage d'une dialogue(infologic.acti.module...,t5,Exécution d'un bouton,Fermeture d'une dialogue,t10,Entrée en saisie dans un formulaire,t10,Affichage d'une dialogue,...,None,None,None,None,None,None,None,None,None,None


Le fichier d'entraînement contient les traces d’utilisation du logiciel Copilote, développé par la société Infologic. Ces traces correspondent à l’enregistrement détaillé des actions réalisées par différents utilisateurs au cours de leurs sessions d’utilisation.

Le jeu de données se présente sous la forme d’un tableau de dimension (3279, 14470), où chaque ligne représente la session d’un utilisateur unique tel que la premiere colonne renseigne l'identifiant de l'utilisateur, la seconde le navigateur puis chacunes des colonnes qui suivent décrivent un événement ou une action observée pendant cette session. L'avancement temporel des actions est également suivi dans le fichier. 
Ces actions couvrent un large éventail de comportements tels que : 
création d’un écran, affichage d’une boîte de dialogue, exécution d’un bouton, saisie dans un champ, filtrage / tri, double-clic, ou encore lancement d’une action générique. 

Le suivi temporel est lui présenté tel que toutes les 5 secondes, une cellule contenant l'avancement (t5, t10,...,t800,...) est inséré après la dernière action effectué durant les 5 secondes précédente.

Certaines colonnes du fichier peuvent être vides (NaN), ce qui traduit l’absence d’action.  
Ainsi, chaque ligne du fichier reflète la chronologie complète d’une session utilisateur, composée d’une succession d’actions structurées et annotées par des patterns décrivant le contexte et le comportement d’utilisation. Ces données serviront ensuite de base à la construction de variables explicatives destinées à entraîner un modèle de classification supervisée, dans le but d’identifier automatiquement l’utilisateur à l’origine d’une trace.

## Understand column category: browsers

In [4]:
# valeurs uniques de la colonne 'navigateur'
navigateurs_uniques = features_train["navigateur"].unique()
print(navigateurs_uniques)
# fréquence de chaque navigateur
freq_navigateurs = features_train["navigateur"].value_counts()
print(freq_navigateurs)


['Firefox' 'Google Chrome' 'Microsoft Edge' 'Opera']
Firefox           1466
Google Chrome     1339
Microsoft Edge     451
Opera               23
Name: navigateur, dtype: int64


Dans un premier temps, nous avons effectué quelques analyses préliminaires afin de nous approprier les données. Nous nous sommes d'abords penchés sur la répartissions des  différents navigateurs parmi les utilisateurs. Cela nous a permis d'une part de confirmer qu'il n'existait que 4 navigateurs différents (Firefox, Google Chrome, Microsoft edge, Opéra) et qu'un n'utilsateur ne changeaient pas de navigateurs d'une session à l'autre. Cela montre que le navigateur est une caractéristique intrasèque à l'utilisateur. 

## Action ratios per session

Dans un second temps, nous avons analysé l'influence de la temporalité dans le jeu de données. Pour cela, nous avons utilisé l'indicateurs suivant: le temps moyen entre chaque actions. 

In [5]:
from visualise.visualise_session_ratios import compute_session_ratios
compute_session_ratios(features_train)

      session_id util navigateur  total_actions   tmax     ratio
3082        3083  wvy    Firefox          11977  11790  1.015861
2812        2813  wvy    Firefox          12219  10625  1.150024
283          284  tmn    Firefox          12314  10100  1.219208
1025        1026  cka    Firefox          12530   9160  1.367904
3234        3235  fxg    Firefox          12530   9080  1.379956


,session_id,util,navigateur,total_actions,tmax,ratio
3082,3083,wvy,Firefox,11977,11790,1.015861
2812,2813,wvy,Firefox,12219,10625,1.150024
283,284,tmn,Firefox,12314,10100,1.219208
1025,1026,cka,Firefox,12530,9160,1.367904
3234,3235,fxg,Firefox,12530,9080,1.379956
...,...,...,...,...,...,...
2200,2201,lgd,Firefox,14463,25,578.520000
2361,2362,avt,Firefox,14465,15,964.333333
1251,1252,fvc,Google Chrome,14465,15,964.333333
664,665,azx,Google Chrome,14466,10,1446.600000


On observe bien une disparité importante en fonction des utilisateurs et des sessions. Il s'agit donc de données différentientes. Par ailleurs, on remarque également que le nombre d'actions par sessions semble relativement stable avec des écarts de nombre d'actions total par sessions relativement négigeable (la sessions avec le moins d'actions ne possède que 17% d'actions en moins que celle qui en possède le plus). La caractéristique intéressante ici est la vitesse d'enchainement des actions qui est propre à un utilisateur. 

## Understand all the user actions

In [6]:
from visualise.visualise_actions import filter_action
import json, os

uniques = features_train.iloc[:, 2:].stack().unique()
filtered_uniques = [filter_action(un) for un in uniques if not str(un).startswith("t")]

filtered_uniques = list(dict.fromkeys(filtered_uniques)) 

# mapping {action: id}
actions_to_id = {act: i for i, act in enumerate(filtered_uniques)}

os.makedirs("./artifacts", exist_ok=True)
with open("./artifacts/actions.json", "w", encoding="utf-8") as f:
    json.dump(actions_to_id, f, ensure_ascii=False, indent=2)

print(filtered_uniques)


["Création d'un écran", "Affichage d'une dialogue", "Exécution d'un bouton", "Fermeture d'une dialogue", 'Double-clic', "Lancement d'une stat", "Affichage d'un toast", "Lancement d'une action générique", 'Filtrage / Tri', 'Saisie dans un champ', "Sélection d'un écran", 'Sélection d’un onglet', "Affichage d'une erreur", 'Action de table', "Clic sur une grille d'historique de recherche", 'Chainage', "Affichage d'une arborescence", 'Retour sur un écran', 'Fermeture de session', 'Raccourci', "Sélection d'un élément", "Désélection d'un élément", "Ouverture d'un panel", "Fermeture d'un panel", "Sélection d'un flag", "Démarrage serveur d'application Tomcat", 'Entrée en saisie dans un formulaire', "Dissimulation d'une arborescence", 'Clic sur une checkbox', 'Clic long', "Lancement d'un tableau de bord", "Lancement d'une action infocentre", "Désélection d'un flag", 'Erreur système grave', "Raccourci dans l'édition de table"]


## Understand all the patterns

In [2]:
from construct.build_patterns import extract_and_save_patterns
from construct.build_browsers import extract_and_save_browsers

pattern_to_id = extract_and_save_patterns(features_train)
navigateurs_to_id = extract_and_save_browsers(features_train)

701 patterns uniques enregistrés dans './artifacts/patterns.txt'
4 navigateurs enregistrés dans './artifacts/browsers.txt'


In [ ]:
from visualise.visualise_patterns import count_unique_pattern_values
pattern_summary = count_unique_pattern_values(features_train)
display(pattern_summary)

,pattern_type,n_unique_values,example_values
0,(),418,infologic.global.modules.GL_PROJET.param.Param...
1,$$,76,"REFWEB, DEVOPS, MAINT, MEM, MOBICRM"
2,<>,219,"mrm24_9544, ACCUEIL_SECU, ADM_5543, Défaut, MT..."


## Tokenise the dataframe

In [2]:
from construct.build_encoded_data import encode_sessions_from_files


encoded_data = encode_sessions_from_files(features_train)

# Aperçu
for session in encoded_data[:3]:
    print(session)

{'user': 'nuh', 'navigateur': 1, 'actions_patterns': [[0, 489], [1], [2], [3], [1], [2], [3], [0, 498], [2, 217], [1], [3], [4], [2], [5, 501], [6], [5, 501], [1, 217], [2], [3], [4], [7], [1], [6], [8], [2], [3], [0], [2, 99, 204], [7], [2, 204], [1, 204], [9, 204], [2, 204], [3, 204], [1, 204], [2, 204], [3, 204], [1, 204], [6], [1, 204], [2, 204], [3, 204], [6], [3, 204], [1, 204], [2, 204], [3, 204], [2, 204], [1, 204], [2, 204], [3, 204], [1, 204], [2, 204], [3, 204], [9, 204], [9, 204], [6], [9, 204], [9, 204], [6], [8], [9, 204], [1, 204], [6], [1, 204], [2, 204], [3, 204], [0, 468], [6], [0, 468], [10, 217, 204], [9, 204], [6], [9, 204], [1, 204], [6], [2, 204], [3, 204], [6], [1, 204], [2, 204], [3, 204], [9, 204], [9, 204], [6], [9, 204], [1, 204], [2, 204], [3, 204], [9, 204], [11, 204], [11, 204], [6], [8], [1, 204], [9, 204], [2, 204], [3, 204], [1, 204], [2, 204], [3, 204], [1, 204], [2, 204], [3, 204], [2, 204], [1, 204], [2, 204], [3, 204], [6], [3, 204], [9, 204], [1, 

In [3]:
from construct.build_actionspatterns import action_pattern_as_columns

action_pattern_as_columns(encoded_data, max_patterns=3)

,user,navigateur,"(0, -1, -1, -1)","(0, 8, -1, -1)","(0, 10, -1, -1)","(0, 11, -1, -1)","(0, 22, -1, -1)","(0, 24, -1, -1)","(0, 25, -1, -1)","(0, 26, -1, -1)","(0, 27, -1, -1)","(0, 28, -1, -1)","(0, 31, -1, -1)","(0, 35, -1, -1)","(0, 37, -1, -1)","(0, 40, -1, -1)","(0, 42, -1, -1)","(0, 45, -1, -1)","(0, 47, -1, -1)","(0, 48, -1, -1)","(0, 49, -1, -1)","(0, 50, -1, -1)","(0, 51, -1, -1)","(0, 51, 25, -1)","(0, 51, 48, -1)","(0, 51, 130, -1)","(0, 51, 211, -1)","(0, 51, 213, -1)","(0, 70, -1, -1)","(0, 71, -1, -1)","(0, 71, 48, -1)","(0, 71, 132, -1)","(0, 71, 206, -1)","(0, 71, 320, -1)","(0, 81, -1, -1)","(0, 82, -1, -1)","(0, 85, -1, -1)","(0, 86, -1, -1)","(0, 87, -1, -1)","(0, 87, 24, -1)",...,"(28, 476, 317, -1)","(28, 500, -1, -1)","(28, 557, 194, -1)","(28, 560, -1, -1)","(28, 560, 366, -1)","(28, 610, 307, -1)","(28, 634, -1, -1)","(28, 640, -1, -1)","(28, 640, 2, -1)","(28, 649, -1, -1)","(28, 667, 306, -1)","(28, 679, -1, -1)","(29, -1, -1, -1)","(29, 123, -1, -1)","(29, 172, -1, -1)","(29, 295, -1, -1)","(29, 303, -1, -1)","(29, 343, -1, -1)","(29, 350, -1, -1)","(29, 370, -1, -1)","(29, 488, -1, -1)","(29, 489, -1, -1)","(29, 496, -1, -1)","(29, 496, 207, -1)","(29, 496, 233, -1)","(29, 496, 312, -1)","(29, 496, 370, -1)","(29, 516, -1, -1)","(29, 522, -1, -1)","(29, 620, -1, -1)","(29, 647, -1, -1)","(30, -1, -1, -1)","(30, 637, -1, -1)","(31, -1, -1, -1)","(31, 427, -1, -1)","(31, 429, -1, -1)","(32, -1, -1, -1)","(33, -1, -1, -1)","(33, 217, -1, -1)","(33, 416, -1, -1)"
0,nuh,1,66,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,muz,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,zrx,3,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,pou,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ald,2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3274,muz,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3275,cjr,2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3276,fuz,1,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3277,cjr,2,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Train and Evaluate

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
def to_categories(df, cols):
    """
    Encode les colonnes catégorielles (comme 'user' ou 'navigateur')
    en entiers compatibles avec XGBoost.
    """
    for col in cols:
        if col in df.columns:
            df[col] = df[col].astype("category").cat.codes
    return df


# Conversion et préparation des données
df_xgb = action_pattern_as_columns(encoded_data, max_patterns=3)
df_xgb = to_categories(df_xgb, ["user", "navigateur"])

# Définir les features et la target
y = df_xgb["user"]
X = df_xgb.drop(columns=["user", "navigateur"])

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Créer DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Paramètres XGBoost pour multi-class
params = {
    "objective": "multi:softprob",  # multi-class
    "eval_metric": "mlogloss",
    "num_class": y.nunique(),
    "max_depth": 6,
    "eta": 0.1,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "seed": 42
}

# Entraînement
num_round = 100
bst = xgb.train(params, dtrain, num_round)

# Prédictions
y_pred = bst.predict(dtest).argmax(axis=1)

# Évaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.7668
